In [ ]:
### Imports 
from pathlib import Path
from sklearn import preprocessing
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Bidirectional, Reshape, TimeDistributed
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from matplotlib import pyplot
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import KFold
from tensorflow.keras import backend as K
from tensorflow.keras.losses import BinaryCrossentropy
from pymms.sdc import mrmms_sdc_api as mms
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime as dt
import os
import time
import sklearn
import scipy
import pickle
import random
import requests
import sys
from imblearn.under_sampling import RandomUnderSampler

###tf.compat.v1.disable_v2_behavior() ### Only for old CUDA version usage

In [ ]:
### Set filepath and import data csv
fpath = 'C:/Users/Davis/Desktop/'
fname = 'MSBrandNewData' + '.csv'
file = fpath+fname
mms_data = pd.read_csv(file, index_col=0, infer_datetime_format=True, parse_dates=[0])

In [ ]:
### Set index and pop off the "selected" column. The T/F value indicates if a SITL selected the timestep. 
### We pop this column because we don't want selections data interpolated a little later
index = mms_data.index
selections = mms_data.pop("selected")
column_names = mms_data.columns

### It's a little later. We rid the data of garbage and interpolate
mms_data = mms_data.replace([np.inf, -np.inf], np.nan)
mms_data = mms_data.interpolate(method='time', limit_area='inside')

### The data is scaled to have zero mean and std dev of 1, courtest of sci-kit learn
scaler = preprocessing.StandardScaler()

### Put everything back together nicely in a Pandas dataframe
mms_data = scaler.fit_transform(mms_data)
mms_data = pd.DataFrame(mms_data, index, column_names)
mms_data = mms_data.join(selections)

In [ ]:
### Because there are far more points not selected than selected, weighting them equally would
### give us garbage out. Thus we weight based on their portion of all points 
false_weight = len(mms_data)/(2*np.bincount(mms_data['selected'].values)[0])
true_weight = len(mms_data)/(2*np.bincount(mms_data['selected'].values)[1])

In [ ]:
### Organize the data into SITL windows, which are contiguous
sitl_windows = mms.mission_events('sroi', mms_data.index[0].to_pydatetime(), mms_data.index[-1].to_pydatetime(), sc='mms1')
windows = []
for start, end in zip(sitl_windows['tstart'], sitl_windows['tend']):
  window = mms_data[start:end]
  if not window.empty and len(window[window['selected']==True])>1:
    windows.append(window)

In [ ]:
### Divide the data into training and testing data, ~70% in train, ~30% in test
while True:
    X_train, X_test, y_train, y_test = [], [], [], []

    sequences = []
    for i in range(len(windows)):
      X_sequence = []
      y_sequence = []

      if random.random() < 0.7:
        for value in windows[i].values:
          X_sequence.append(value[:-1])
          y_sequence.append(value[-1])
          if len(X_sequence) == SEQ_LEN:
            X_train.append(X_sequence.copy())
            
            y_train.append(y_sequence.copy())

            X_sequence = []
            y_sequence = []

      else:
        for value in windows[i].values:
          X_sequence.append(value[:-1])
          y_sequence.append(value[-1])
          if len(X_sequence) == SEQ_LEN:
            X_test.append(X_sequence.copy())
            
            y_test.append(y_sequence.copy())

            X_sequence = []
            y_sequence = []

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.expand_dims(np.array(y_train), axis=2)
    y_test = np.expand_dims(np.array(y_test), axis=2)

    if len(X_train) > len(X_test):
        break

In [ ]:
### Check how organizing went
print(f"Number of sequences in training data: {len(X_train)}")
print(f"Number of sequences in test data: {len(X_test)}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
### Calculate F1 score
# (Credit: Paddy and Kev1n91 from https://stackoverflow.com/a/45305384/3988976)
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
### We use weighted binary crossentropy as it gives more weight to positive classes when large amount of outputs are zero
# (Credit: tobigue from https://stackoverflow.com/questions/42158866/neural-network-for-multi-label-classification-with-large-number-of-classes-outpu)
def weighted_binary_crossentropy(target, output):
    """
    Weighted binary crossentropy between an output tensor 
    and a target tensor. POS_WEIGHT is used as a multiplier 
    for the positive targets.

    Combination of the following functions:
    * keras.losses.binary_crossentropy
    * keras.backend.tensorflow_backend.binary_crossentropy
    * tf.nn.weighted_cross_entropy_with_logits
    """
     # transform back to logits
    _epsilon = tf.convert_to_tensor(K.epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
    output = tf.math.log(output / (1 - output))
    # compute weighted loss
    #target = tf.cast(target)
    loss = tf.nn.weighted_cross_entropy_with_logits(labels=target,
                                                    logits=output,
                                                    pos_weight=true_weight)
    return tf.reduce_mean(loss, axis=-1)

In [ ]:
### Epochs, batch size, sequence legnth
EPOCHS = 200
BATCH_SIZE = 72
SEQ_LEN = 250

In [ ]:
### Set up our model

model_name = f"{SEQ_LEN}-SEQ_LEN-{BATCH_SIZE}-BATCH_SIZE-{LAYER_SIZE}-LAYER_SIZE-{int(time.time())}"

model = Sequential()

model.add(Bidirectional(LSTM(LAYER_SIZE, return_sequences=True), input_shape= (None, X_train.shape[2])))

model.add(Dropout(0.3))

model.add(Bidirectional(LSTM(LAYER_SIZE, return_sequences=True), input_shape= (None, X_train.shape[2])))

model.add(TimeDistributed(Dense(1, activation='sigmoid')))

opt = tf.keras.optimizers.SGD()

model.compile(loss = weighted_binary_crossentropy,
        optimizer=opt,
        metrics=['accuracy', f1, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()    

In [ ]:
### Set filepath and model training checkpoints
filepath = "C:/Users/Davis/Desktop/ms-dl-davisMSSGDv27.model"
checkpoint = ModelCheckpoint(filepath, monitor='val_f1', verbose=1, save_best_only=True, mode='max')

In [ ]:
### Train and test the model
history = model.fit(
  x=X_train, y=y_train,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_data=(X_test, y_test),
  callbacks=[checkpoint],
  verbose=1,
  shuffle=True)

In [ ]:
### Plot loss of the model on training and testing data as a function of training epoch
plt.plot(history.history['loss']) ### training
plt.plot(history.history['val_loss'])  ### testing
plt.title('Model Training Loss vs. Testing Loss by Epoch')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'testing'], loc='upper right')
plt.show()

In [ ]:
### Plot F1 score on training and testing data as a function of training epoch
plt.plot(history.history['f1'])
plt.plot(history.history['val_f1'])
plt.title('Model Training F1 vs. Testing F1 by Epoch')
plt.ylabel('F1')
plt.xlabel('Epoch')
plt.legend(['train', 'testing'], loc='upper right')
plt.show()

In [ ]:
### Plot precision on training and testing data as a function of training epoch
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('Model Training Precision vs. Testing Precision by Epoch')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['train', 'testing'], loc='upper right')
plt.show()

In [ ]:
### Plot model accuracy on training and testing data as a function of training epoch
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Training accuracy vs. accuracy by Epoch')
plt.ylabel('acc')
plt.xlabel('Epoch')
plt.legend(['train', 'testing'], loc='upper right')
plt.show()

In [ ]:
### Load validation data
validation_data = pd.read_csv('C:/Users/Davis/Desktop/MSNewData.csv', index_col=0, infer_datetime_format=True, parse_dates=[0])

In [ ]:
### Load the model
model = tf.keras.models.load_model('C:/Users/Davis/Desktop/ms-dl-davisMSSGDv27.model', {'weighted_binary_crossentropy':weighted_binary_crossentropy, 'f1':f1})

In [ ]:
### Preprocess validation data in the same way as training/testing data previously
scaler = preprocessing.StandardScaler()
index = validation_data.index
selections = validation_data.pop('selected')
column_names = validation_data.columns

validation_data = validation_data.replace([np.inf, -np.inf], np.nan)
validation_data = validation_data.interpolate(method='time', limit_area='inside')

validation_data = scaler.fit_transform(validation_data)
validation_data = pd.DataFrame(validation_data, index, column_names)
validation_data = validation_data.join(selections)

In [ ]:
### Assign data and selections to X and Y, respectively
validation_X = np.asarray(validation_data.values[:,:-1])
validation_y = np.asarray(validation_data.values[:,-1])

### TensorFlow doesn't agree with the data unless it is reshaped
validation_X = validation_X[:, np.newaxis, :]
validation_y = validation_y[:, np.newaxis]

In [ ]:
### Test the model
test_predictions = model.predict(validation_X)

In [ ]:
desktop = 'C:/Users/Davis/Desktop/' ### Filepath declaration

### Plot Ground Truth predictions
plt.figure(figsize=(28, 5))
plt.plot(validation_y.astype(int))
plt.title("Ground Truth (SITL) Selections by Datapoint - Magnetosphere")
plt.ylabel('Selected (1) or not (0)')
plt.xlabel('Datapoint')
plt.savefig(desktop + 'SWSITL.pdf')
plt.show()

In [ ]:
### Plot model's predictions
plt.figure(figsize=(28, 5))
plt.plot(test_predictions.flatten())
plt.title("Model Predicted Selections by Datapoint - Solar Wind")
plt.ylabel('Selection confidence (continous)')
plt.xlabel('Datapoint')
plt.savefig(desktop + 'SWModel.pdf')
plt.show()


In [ ]:
### Plot only predictions made with greater than 50% confidence
t_output = [0 if x < 0.5 else 1 for x in test_predictions.flatten()]
plt.figure(figsize=(28, 5))
plt.plot(t_output)
plt.title("Filtered Model Predictions by Datapoint - Solar Wind")
plt.ylabel('Selected (1) or not (0)')
plt.xlabel('Datapoints')
plt.savefig(desktop + 'SWFiltered.pdf')
plt.show()

In [ ]:
### Preprocess data for ROC 
y_eval = validation_y.astype(int)
y_eval = y_eval.flatten()

y_true = (np.asarray(t_output)).squeeze()
y_true = y_true.flatten()

In [ ]:
### Create ROC
plt.figure(figsize = (10, 6))
fpr, tpr, thresholds = roc_curve(y_eval, y_true)
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Solar Wind ROC curve - AUC = {:.2f}'.format(auc(fpr, tpr)))
plt.legend(loc="lower right")
plt.savefig(desktop + 'SWROCCurve.pdf')

plt.show()

In [ ]:
### Create confusion matrix
cm = confusion_matrix(y_eval, t_output)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()